In [1]:
import datetime

import numpy as np
import pandas as pd
import re
import datetime
from sklearn.preprocessing import OneHotEncoder

In [2]:
package_file = 'package.csv'
product_file = 'product.csv'

product_data = pd.read_csv(product_file, sep=';', encoding='latin1')
package_data = pd.read_csv(package_file, sep=';', encoding='latin1')

product_data_description = product_data.describe()
print(product_data_description)

# TODO incohernce entre dates
# TODO incohernce entre routname / forme
# TODO incohernce entre valeurs numeric abberantes (ordre de grandeur)
# TODO incohernce entre valeurs phase et l'emballage
# TODO tester imputatin itérative
# TODO utiliser le one hot de sk learn au lieu de dummies de pandas

def assert_product_id_completeness(table, header):

    empty_cells = table.shape[0] - table.count(axis=0)
    unique_values = table.nunique(axis=0)

    print('Empty cells:\n{}\n'.format(empty_cells))
    print('Unique values:\n{}\n'.format(unique_values))

    try:
        assert empty_cells[header] == 0
        print('No empty cells in {} column'.format(header))
    except:
        print('There are {} empty values in the {} column'.format(empty_cells[header], header))
    try:
        assert unique_values[header] == table.shape[0]
        print('No duplicat values in the {} column'.format(header))
    except:
        print('There are {} duplicat values in the {} column'.format(table.shape[0] - unique_values[header], header))

print('Assessing completnes of PRODUCTID for product data')
assert_product_id_completeness(product_data, 'PRODUCTID')

print('Assessing completnes of PRODUCTID for packaging data')
assert_product_id_completeness(package_data, 'PRODUCTID')

       STARTMARKETINGDATE  ENDMARKETINGDATE  LISTING_RECORD_CERTIFIED_THROUGH
count        9.323800e+04      4.323000e+03                      8.891300e+04
mean         2.010696e+07      2.021040e+07                      2.020247e+07
std          1.131769e+05      1.539891e+05                      3.294780e+03
min          1.900010e+07      2.020022e+07                      2.020123e+07
25%          2.009090e+07      2.020081e+07                      2.020123e+07
50%          2.014122e+07      2.020123e+07                      2.020123e+07
75%          2.018010e+07      2.021111e+07                      2.020123e+07
max          2.020021e+07      3.031021e+07                      2.021123e+07
Assessing completnes of PRODUCTID for product data
Empty cells:
PRODUCTID                               0
PRODUCTNDC                              0
PRODUCTTYPENAME                         0
PROPRIETARYNAME                         6
PROPRIETARYNAMESUFFIX               83075
NONPROPRIETARYNAME      

# 1. Auscultation
## Etude des données du fichier 'package'

In [3]:
package_data.head()

,PRODUCTID,PRODUCTNDC,NDCPACKAGECODE,PACKAGEDESCRIPTION,STARTMARKETINGDATE,ENDMARKETINGDATE,NDC_EXCLUDE_FLAG,SAMPLE_PACKAGE
0,0699-0155_60f08fe4-aacf-c815-e053-2991aa0a3ded,0699-0155,0699-0155-44,1 CANISTER in 1 CARTON (0699-0155-44) > 125 g...,19000101.0,NaN,N,N
1,0699-0311_59039cf6-9008-7367-e053-2a91aa0a2fe9,0699-0311,0699-0311-16,"498 g in 1 BOTTLE, PLASTIC (0699-0311-16)",19000101.0,NaN,N,N
2,0699-0312_59039cf6-9008-7367-e053-2a91aa0a2fe9,0699-0312,0699-0312-16,"498 g in 1 BOTTLE, PLASTIC (0699-0312-16)",19000101.0,NaN,N,N
3,0699-0317_59039cf6-9008-7367-e053-2a91aa0a2fe9,0699-0317,0699-0317-16,"498 g in 1 BOTTLE, PLASTIC (0699-0317-16)",19000101.0,NaN,N,N
4,0699-0404_5bc1c974-e7b8-094c-e053-2991aa0ab726,0699-0404,0699-0404-01,28 g in 1 JAR (0699-0404-01),19000101.0,NaN,N,N


In [4]:
count_missing_values_package = package_data.isnull().sum().sort_values()
# TODO: count uniques values for each column
# %%
"""
### Colonne PACKAGEDESCRIPTION
Cette colonne est présentée sous forme de phrase et contient de multiples informations. Le volume, son unité, le nombre 
de contenant et son type. S'il existe plusieurs contenants pour un objet, ils sont concaténés par un séparateur '>' de 
manière hiérarchique.
"""

"\n### Colonne PACKAGEDESCRIPTION\nCette colonne est présentée sous forme de phrase et contient de multiples informations. Le volume, son unité, le nombre \nde contenant et son type. S'il existe plusieurs contenants pour un objet, ils sont concaténés par un séparateur '>' de \nmanière hiérarchique.\n"

### Colonne STARTMARKETINGDATE
Les valeurs de cette colonne sont de type date. Elles ont l'air séquentielles. 

### Colonne ENDMARKETINGDATE
Les valeurs de cette colonne sont de type date. Il existe beaucoup de valeurs manquantes.

### Colonne NDC_EXCLUDE_FLAG

In [5]:
package_data['NDC_EXCLUDE_FLAG'].unique()

array(['N'], dtype=object)

Tous les objets possèdent la valeur N et ne présente aucune valeur manquante. 

### Colonne SAMPLE_PACKAGE

In [6]:
package_data['SAMPLE_PACKAGE'].value_counts()

N    173223
Y       664
Name: SAMPLE_PACKAGE, dtype: int64

Les valeurs possibles sont 'Y' ou 'N'. Il y a une majorité de 'N' et aucune valeur manquante.

## Etude des données du fichier 'product'

In [7]:
product_data.head()

,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
0,0699-0155_60f08fe4-aacf-c815-e053-2991aa0a3ded,0699-0155,HUMAN PRESCRIPTION DRUG,Topex Neutral Fluoride Foam,NaN,sodium fluoride,"AEROSOL, FOAM",DENTAL,19000101,NaN,UNAPPROVED DRUG OTHER,NaN,Dentsply LLC. Professional Division Trading as...,SODIUM FLUORIDE,.02,g/g,NaN,NaN,N,20201231.0
1,0699-0311_59039cf6-9008-7367-e053-2a91aa0a2fe9,0699-0311,HUMAN PRESCRIPTION DRUG,Topex 60 Second Fluoride Gel,NaN,Sodium Fluoride,GEL,DENTAL,19000101,NaN,UNAPPROVED DRUG OTHER,NaN,Dentsply LLC. Professional Division Trading as...,SODIUM FLUORIDE,25.9,mg/g,NaN,NaN,N,20201231.0
2,0699-0312_59039cf6-9008-7367-e053-2a91aa0a2fe9,0699-0312,HUMAN PRESCRIPTION DRUG,Topex 60 Second Fluoride Gel,NaN,Sodium Fluoride,GEL,DENTAL,19000101,NaN,UNAPPROVED DRUG OTHER,NaN,Dentsply LLC. Professional Division Trading as...,SODIUM FLUORIDE,25.9,mg/g,NaN,NaN,N,20201231.0
3,0699-0317_59039cf6-9008-7367-e053-2a91aa0a2fe9,0699-0317,HUMAN PRESCRIPTION DRUG,Topex 60 Second Fluoride Gel,NaN,Sodium Fluoride,GEL,DENTAL,19000101,NaN,UNAPPROVED DRUG OTHER,NaN,Dentsply LLC. Professional Division Trading as...,SODIUM FLUORIDE,25.9,mg/g,NaN,NaN,N,20201231.0
4,0699-0404_5bc1c974-e7b8-094c-e053-2991aa0ab726,0699-0404,HUMAN PRESCRIPTION DRUG,Dry Socket,NaN,Guaiacol and Eugenol,PASTE,DENTAL,19000101,NaN,UNAPPROVED DRUG OTHER,NaN,"Dentsply LLC, Professional Division, trading a...",GUAIACOL; EUGENOL,.0416; .0416,g/g; g/g,"Standardized Chemical Allergen [EPC],Increased...",NaN,N,20201231.0


In [8]:
count_missing_values_product = product_data.isnull().sum().sort_values()
# TODO: count uniques values for each column

### Colonne PRODUCTTYPENAME

In [9]:
product_data['PRODUCTTYPENAME'].value_counts()

HUMAN OTC DRUG                 46172
HUMAN PRESCRIPTION DRUG        44526
NON-STANDARDIZED ALLERGENIC     2008
PLASMA DERIVATIVE                294
STANDARDIZED ALLERGENIC          124
VACCINE                          108
CELLULAR THERAPY                   6
Name: PRODUCTTYPENAME, dtype: int64

Il y a 7 valeurs possibles textuelles catégorielles dans cette colonne.

### Colonne PROPRIETARYNAME

In [10]:
product_data['PROPRIETARYNAME'].nunique()

32716

In [11]:
product_data['PROPRIETARYNAME'][393:401]

393    SUGAR FREE ORIGINAL SWISS HERB COUGH SUPPRESSA...
394    Sugar Free Mountain Herb Cough Suppressant Thr...
395                                              Albuked
396                                              Demerol
397                                              Demerol
398                                              Desoxyn
399                                               OXYGEN
400                                               Oxygen
Name: PROPRIETARYNAME, dtype: object

Dans cette colonne, il existe un grand nombre de valeurs différentes. Les mêmes valeurs peuvent être présentes sous 
différentes formes notamment en minuscules ou majuscules, il existe donc une inconsistance entre les valeurs.

### Colonne PROPRIETARYNAMESUFFIX

In [12]:
product_data['PROPRIETARYNAMESUFFIX'].nunique()

4022

Dans cette colonne, il y a un nombre important de valeurs manquantes. Ces valeurs sont textuelles et il existe un nombre 
élevé de valeurs différentes.

### Colonne NONPROPRIETARYNAME

In [13]:
product_data['NONPROPRIETARYNAME'].nunique()

16257

In [14]:
product_data['NONPROPRIETARYNAME'][2:6]

2         Sodium Fluoride
3         Sodium Fluoride
4    Guaiacol and Eugenol
5         sodium fluoride
Name: NONPROPRIETARYNAME, dtype: object

Cette colonne présente seulement 4 valeurs manquantes. Ceux sont des données textuelles inconsistantes, par exemple 
pouvant représenter la même valeur en caratères minuscules ou majuscules. Il y a un nombre très important de valeurs 
différentes.

### Colonne DOSAGEFORMNAME

In [15]:
product_data['DOSAGEFORMNAME'].value_counts()

TABLET                              15442
TABLET, FILM COATED                  9387
LIQUID                               8764
CREAM                                5104
CAPSULE                              4824
                                    ...  
CHEWABLE GEL                            1
SOLUTION, GEL FORMING / DROPS           1
CRYSTAL                                 1
FOR SUSPENSION, EXTENDED RELEASE        1
INJECTION, LIPID COMPLEX                1
Name: DOSAGEFORMNAME, Length: 134, dtype: int64

Cette colonne contient 134 différentes valeurs textuelles. Comme on peut le voir, différentes catégories peuvent être 
affectées au même objet. La colonne ne présente aucune valeur manquante.

### Colonne ROUTENAME

In [16]:
product_data['ROUTENAME'].value_counts()

ORAL                                                                                54704
TOPICAL                                                                             21547
INTRAVENOUS                                                                          3374
INTRADERMAL; PERCUTANEOUS; SUBCUTANEOUS                                              1315
RESPIRATORY (INHALATION)                                                             1257
                                                                                    ...  
INTRAVENOUS; INTRAVESICAL; OPHTHALMIC                                                   1
SUBGINGIVAL                                                                             1
INTRACORONARY; RESPIRATORY (INHALATION)                                                 1
INFILTRATION; INTRA-ARTICULAR; INTRALESIONAL; INTRAMUSCULAR; PERINEURAL; TOPICAL        1
EPIDURAL; INFILTRATION; INTRAMUSCULAR; INTRASYNOVIAL; SOFT TISSUE; TOPICAL              1
Name: ROUT

Cette colonne contient 180 différentes valeurs textuelles. Comme on peut le voir, différentes catégories peuvent être 
affectées au même objet. La colonne présente 1932 valeurs manquantes.

### Colonne STARTMARKETINGDATE
Les valeurs sont de type date, il n'y a aucune valeur manquante. 

### Colonne ENDMARKETINGDATE
Les valeurs sont de type date, il y a un grand nombre de valeurs manquantes. 

### Colonne MARKETINGCATEGORYNAME

In [17]:
product_data['MARKETINGCATEGORYNAME'].value_counts()
# %%
"""
Les valeurs sont de type textuelles, il y a 26 catégories différentes et ne présente aucune valeur manquante.  
"""
# %%
"""
### Colonne APPLICATIONNUMBER
"""
# %%
product_data['APPLICATIONNUMBER'].nunique()
# %%
"""
Cette colonne spécifie le numéro de série de la catégorie marketing. Le nombre de valeurs manquantes est élevé, et 
comme il y a un numéro de série pour chaque objet dans une catégorie, le nombre de valeurs différentes est également 
important.
"""

'\nCette colonne spécifie le numéro de série de la catégorie marketing. Le nombre de valeurs manquantes est élevé, et \ncomme il y a un numéro de série pour chaque objet dans une catégorie, le nombre de valeurs différentes est également \nimportant.\n'

### Colonne LABELERNAME

In [18]:
product_data['LABELERNAME'].nunique()

6606

In [19]:
product_data['LABELERNAME'][7291:7293]
# %%
"""
La colonne présente peu de valeurs manquantes (557). Si on remarque qu'il existe un nombre important de valeurs 
différentes, les données sont cependant inconsistantes.
"""

"\nLa colonne présente peu de valeurs manquantes (557). Si on remarque qu'il existe un nombre important de valeurs \ndifférentes, les données sont cependant inconsistantes.\n"

### Colonne SUBSTANCENAME

In [20]:
product_data['SUBSTANCENAME'].nunique()

8976

In [21]:
product_data['SUBSTANCENAME'][727:735]

727    XANTHIUM STRUMARIUM POLLEN; PLANTAGO LANCEOLAT...
728    XANTHIUM STRUMARIUM POLLEN; PLANTAGO LANCEOLAT...
729    ALNUS RHOMBIFOLIA POLLEN; FRAXINUS AMERICANA P...
730    ALNUS RHOMBIFOLIA POLLEN; FRAXINUS AMERICANA P...
731    ALNUS RHOMBIFOLIA POLLEN; FRAXINUS AMERICANA P...
732       BETULA NIGRA POLLEN; BETULA POPULIFOLIA POLLEN
733    GIBBERELLA ZEAE; ASPERGILLUS NIGER VAR. NIGER;...
734       CARYA OVATA POLLEN; CARYA ILLINOINENSIS POLLEN
Name: SUBSTANCENAME, dtype: object

Cette colonne présente un nombre assez important de données manquantes (2309), ceux sont des données textuelles 
catégorielles. Or le nombre de catégories parait élevé, comme le montre le nombre de valeurs uniques. Chaque objet 
peut cependant présenté plusieurs catégories séparées par un ';'.

### Colonne ACTIVE_NUMERATOR_STRENGTH

In [22]:
product_data['ACTIVE_NUMERATOR_STRENGTH'][725:731]

725                                                  .05
726                                                   .1
727                                   .05; .05; .05; .05
728                           40000; 40000; 40000; 40000
729          .05; .05; .05; .05; .05; .05; .05; .05; .05
730    40000; 40000; 40000; 40000; 40000; 40000; 4000...
Name: ACTIVE_NUMERATOR_STRENGTH, dtype: object

Ceux sont des données numériques qui paraissent dupliquées pour le même objet.  

### Colonne ACTIVE_INGRED_UNIT

In [23]:
product_data['ACTIVE_INGRED_UNIT'][725:731]

725                                                 g/mL
726                                                 g/mL
727                               g/mL; g/mL; g/mL; g/mL
728               [PNU]/mL; [PNU]/mL; [PNU]/mL; [PNU]/mL
729    g/mL; g/mL; g/mL; g/mL; g/mL; g/mL; g/mL; g/mL...
730    [PNU]/mL; [PNU]/mL; [PNU]/mL; [PNU]/mL; [PNU]/...
Name: ACTIVE_INGRED_UNIT, dtype: object

Ceux sont des données textuelles catégorielles qui présentent l'unité de la colonne 'ACTIVE_INGRED_UNIT'. Les données
paraissent également dupliquées pour le même objet.

### Colonne PHARM_CLASSES

In [24]:
product_data['PHARM_CLASSES'].nunique()

1285

In [25]:
product_data['PHARM_CLASSES'][725]

'Non-Standardized Pollen Allergenic Extract [EPC],Increased Histamine Release [PE],Cell-mediated Immunity [PE],Increased IgG Production [PE],Pollen [CS],Allergens [CS]'

Ceux sont des données textuelles catégorielles présentant plusieurs catégories pour un même objet. Il y a un grand 
nombre de valeurs manquantes.
"""
# %%
"""
### Colonne DEASCHEDULE

In [26]:
product_data['DEASCHEDULE'].value_counts()

CII     1802
CIV     1749
CIII     479
CV       393
Name: DEASCHEDULE, dtype: int64

Cette colonne présente un nombre important de données manquantes. Ceux sont des données catégorielles, présentant 
seulement 4 catégories.

### Colonne NDC_EXCLUDE_FLAG

In [27]:
product_data['NDC_EXCLUDE_FLAG'].value_counts()

N    93238
Name: NDC_EXCLUDE_FLAG, dtype: int64

Cette colonne présente seulement une catégorie 'N'.

# 2. Relations entre attributs
## Informations communes
Les colonnes 'PRODUCTID' des tables 'package' et 'product' contiennent deux informations concaténées: l'id du produit 
ainsi que le contenu de leur colonne 'PRODUCTNDC', le code label et le code segment produit.  
Dans la documentation NDC, il est précisé que c'est pour prévenir le duplicata de lignes.

La colonne 'NDCPACKAGECODE' de la table 'package' contient deux informations concaténées: le code segment du package et 
le contenu de la colonne 'PRODUCTNDC', le code label et le code segment produit.

La colonne 'PACKAGEDESCRIPTION' de la table 'package' contient plusieurs informations concaténées. En plus des 
informations propres à la description du package, il y a dans la majorité des objets la valeur 'NDCPACKAGECODE' associée
.

La colonne 'APPLICATIONNUMBER' de la table 'product' présente la majorité du temps le contenu de la colonne 
'MARKETINGCATEGORYNAME' et spécifie son numéro de série.

Dans les deux tables, il existe des colonnes 'STARTMARKETINGDATE',  'ENDMARKETINGDATE' et 'NDCEXLUDEDFLAG'. 
Elles semblent présenter les mêmes informations.

## Corrélation
Pour la table 'product':
Il semble pouvoir exister une corrélation entre les attributs 'ROUTENAME' et 'DOSAGEFORMNAME' qui présentent des idées 
d'administration similaires. 
On peut également considérer l'existance d'une corrélation entre les modes d'administration
et les dosages du médicament, donc les attributs 'ROUTENAME', 'DOSAGEFORMNAME' et ceux 'ACTIVE_NUMERATOR_STRENGTH', 
'ACTIVE_INGRED_UNIT'.
L'attribut 'PHARM_CLASS' semble pouvoir être corrélé à l'attribut 'SUBSTANCENAME'.

# 3. Correction des incohérences
On élimine dans un premier temps les duplicata de valeurs dans les attributs 'ACTIVE_NUMERATOR_STRENGTH', 
'ACTIVE_INGRED_UNIT' de la table 'product'. 
## Table 'product'

In [28]:

# TODO: keep most frequent value
dupl_val_cols = ['ACTIVE_NUMERATOR_STRENGTH', 'ACTIVE_INGRED_UNIT']
for c in dupl_val_cols:
    product_data[c] = product_data[c].replace(to_replace=r'\;.*', value='', regex=True)

On tranforme toutes les valeurs textuelles des différents attributs.en charactères minuscule

In [29]:
cols_to_lower = ['PRODUCTTYPENAME', 'PROPRIETARYNAME', 'PROPRIETARYNAMESUFFIX', 'NONPROPRIETARYNAME',
                     'DOSAGEFORMNAME', 'ROUTENAME', 'MARKETINGCATEGORYNAME', 'APPLICATIONNUMBER',
                     'LABELERNAME', 'SUBSTANCENAME', 'ACTIVE_INGRED_UNIT', 'PHARM_CLASSES', 'DEASCHEDULE']
for c in cols_to_lower:
    product_data[c] = product_data[c].str.lower()

Il existerait également une incohérence si l'attribut 'ENDMARKETINGDATE' est moins récent que le 'STARTMARKETINGDATE'.
On vérifie s'il en existe dans les tables 'product' et 'package'.

In [30]:

# conversion to datetime format
date_cols = ['STARTMARKETINGDATE', 'ENDMARKETINGDATE', 'LISTING_RECORD_CERTIFIED_THROUGH']
for c in date_cols:
    product_data[c] = pd.to_datetime(product_data[c], errors='coerce', format='%Y%m%d')

# compare STARTMARKETINGDATE and ENDMARKETINGDATE
# replace ENDMARKETINGDATE to NaT when incoherence
product_data.loc[
    (product_data['STARTMARKETINGDATE'] > product_data['ENDMARKETINGDATE']), 'ENDMARKETINGDATE'] = pd.NaT

## Table 'package'
La colonne 'PACKAGEDESCRIPTION' contient beaucoup trop d'informations pour être exploitable. Tout d'abord, on garde 
seulement l'information du package le plus informatif (le dernier) car spécifie le volume le plus précis.
On supprime l'information dupliquée du 'NDCPACKAGECODE'. 
Enfin, on crée une colonne pour chaque information: 'PACKAGESIZE', 'PACKAGEUNIT' et 'PACKAGETYPE'.
On peut retirer la colonne 'PACKAGEDESCRIPTION' de la table.

In [31]:
# keep only most informative packaging and remove duplicate info NDCPACKAGECODE
package_data['PACKAGEDESCRIPTION'] = package_data['PACKAGEDESCRIPTION'].replace(to_replace=r'.*(\>|\*\ ) |\(.*',
                                                                                value='', regex=True)

# split info into multiple columns
search = {0: [], 1: [], 2: []}
for values in package_data['PACKAGEDESCRIPTION']:
    s = re.search(r'(^\.?[0-9\.]+)\ (.*)\ in\ 1\ (.*)', values)
    for i in range(3):
        search[i].append(s.group(i + 1))

for i, n in enumerate(['PACKAGESIZE', 'PACKAGEUNIT', 'PACKAGETYPE']):
    package_data[n] = search[i]

Traitement des colonnes 'STARTMARKETINGDATE', 'ENDMARKETINGDATE' similairement à la table 'product'.

In [32]:
# conversion to datetime format
date_cols = ['STARTMARKETINGDATE', 'ENDMARKETINGDATE']
for c in date_cols:
    package_data[c] = pd.to_datetime(package_data[c], errors='coerce', format='%Y%m%d')

# compare STARTMARKETINGDATE and ENDMARKETINGDATE
# replace ENDMARKETINGDATE to NaT when incoherence
package_data.loc[
    (package_data['STARTMARKETINGDATE'] > package_data['ENDMARKETINGDATE']), 'ENDMARKETINGDATE'] = pd.NaT

# 4. Données manquantes
## Table 'package'
On s'intéresse aux données manquantes dans les colonnes PRODUCTID, PRODUCTNDC, NDCPACKAGECODE.

In [33]:
package_missing_ndcpackagecode = package_data.iloc[np.where(pd.isnull(package_data['NDCPACKAGECODE']))]
values = package_missing_ndcpackagecode['PACKAGEDESCRIPTION'].str.extract(r'\((.*?)\).*')
for index, row in values.iterrows():
    package_data.loc[index, 'NDCPACKAGECODE'] = row[0]

In [34]:
package_missing_productndc = package_data.iloc[np.where(pd.isnull(package_data['PRODUCTNDC']))]
values = package_missing_productndc['NDCPACKAGECODE'].str.extract(r'^([\w]+-[\w]+)')
for index, row in values.iterrows():
    package_data.loc[index, 'PRODUCTNDC'] = row[0]

In [35]:
# TODO : find a way to retrieve PRODUCTID from 'product' table
package_missing_ndcproductid = package_data.iloc[np.where(pd.isnull(package_data['PRODUCTID']))]

Il existe des valeurs manquantes pour les colonnes 'STARTMARKETINGDATE' et 'ENDMARKETINGDATE' dans la table 'package'
mais on choisit de ne pas les compléter car on ne peut effectuer d'estimation précise. 

## Table 'product'

# 5. Duplications données

In [36]:

# TODO: drop column PACKAGEDESCRIPTION

# Transformation en données numériques (après question 8)
## Table 'package'

In [37]:

transf_package_data = package_data

# TODO: change get_dummies to OneHotEncoder
# transform PACKAGEUNIT and PACKAGETYPE categorial columns (multiple values) to one hot
transf_package_data = pd.concat([transf_package_data, transf_package_data['PACKAGEUNIT']
                                .str.get_dummies(sep=', ')
                                .add_prefix('PACKAGEUNIT')], axis=1)
transf_package_data = transf_package_data.drop(columns=['PACKAGEUNIT'])
transf_package_data = pd.concat([transf_package_data, transf_package_data['PACKAGETYPE']
                                .str.get_dummies(sep=', ')
                                .add_prefix('PACKAGETYPE')], axis=1)
transf_package_data = transf_package_data.drop(columns=['PACKAGETYPE'])

In [38]:

# convert PACKAGESIZE to proper numerical value
transf_package_data['PACKAGESIZE'] = pd.to_numeric(transf_package_data['PACKAGESIZE'])

In [39]:

# TODO: change get_dummies to OneHotEncoder
# convert NDC_EXCLUDE_FLAG and SAMPLE_PACKAGE to one hot
transf_package_data = pd.get_dummies(data=transf_package_data, columns=['NDC_EXCLUDE_FLAG', 'SAMPLE_PACKAGE'])

## Table 'product'

In [40]:

transf_product_data = product_data

# # transform ROUTENAME and DOSAGEFORMNAME categorial columns (multiple values) to one hot
# transf_product_data = pd.concat([transf_product_data, transf_product_data['ROUTENAME']
#                                 .str.get_dummies(sep='; ')
#                                 .add_prefix('ROUTENAME')], axis=1)
# transf_product_data = transf_product_data.drop(columns=['ROUTENAME'])
# transf_product_data = pd.concat([transf_product_data, transf_product_data['DOSAGEFORMNAME']
#                                 .str.get_dummies(sep=', ')
#                                 .add_prefix('DOSAGEFORMNAME')], axis=1)
# transf_product_data = transf_product_data.drop(columns=['DOSAGEFORMNAME'])
#
# # %%
#
# # convert PRODUCTTYPENAME, NDC_EXCLUDE_FLAG to one hot
# transf_product_data = pd.get_dummies(data=transf_product_data, columns=['PRODUCTTYPENAME', 'NDC_EXCLUDE_FLAG'])
#
# # convert ACTIVE_NUMERATOR_STRENGTH to proper numerical value
# transf_product_data['ACTIVE_NUMERATOR_STRENGTH'] = pd.to_numeric(transf_product_data['ACTIVE_NUMERATOR_STRENGTH'])

In [41]:
# TODO : one hot MARKETINGCATEGORYNAME
# TODO: hash PROPRIETARYNAME NONPROPRIETARYNAME LABELERNAME PROPRIETARYNAMESUFFIX
# TODO: separate and hash SUBSTANCENAME PHARM_CLASSES
# TODO : split ACTIVE_INGRED_UNIT by '/' (nan others), then one hot each col

# TODO: ideas?? APPLICATIONNUMBER
# %%
# TODO : analysis ratio per category

# Save transformed data to file
transf_product_data.to_csv('transformed_product_data.csv', sep='\t', encoding='utf-8')
transf_package_data.to_csv('transformed_package_data.csv', sep='\t', encoding='utf-8')